# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [1]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
from newsapi.newsapi_client import NewsApiClient
from datetime import datetime, timedelta
import alpaca_trade_api as tradeapi
%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Dev\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# Read your api key environment variable
# Init
# Load .env enviroment variables
load_dotenv()
# Set News API Key
newsapi = NewsApiClient(api_key=os.environ["NEWS_API"])
analyzer= SentimentIntensityAnalyzer()
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Dev\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
# Fetch the Bitcoin news articles
btc_headlines =newsapi.get_everything(
q="bitcoin", 
    language ="en",
    page_size= 100,
    sort_by= "relevancy"
    
)

In [4]:
# Fetch the Ethereum news articles
eth_headlines =newsapi.get_everything(
q="ethereum", 
    language ="en",
    page_size= 100,
    sort_by= "relevancy"
    
)

In [5]:
eth_sentiments = []

for x in eth_headlines["articles"]:
    try:
        # iterate through each element in headlines
        text = x["content"]
        date = x["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text) # VADER sentiment scores are retrieved
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        # append each element of each iteration to the libra_sentiments empty list
        eth_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
          })
    except AttributeError:
        pass

In [6]:
btc_sentiments = []

for x in btc_headlines["articles"]:
    try:
        # iterate through each element in headlines
        text = x["content"]
        date = x["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text) # VADER sentiment scores are retrieved
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        # append each element of each iteration to the libra_sentiments empty list
        btc_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
          })
    except AttributeError:
        pass

In [7]:
# Create the Bitcoin sentiment scores DataFrame

btc_df=pd.DataFrame(btc_sentiments)

In [8]:
# Create the Ethereum sentiment scores DataFrame

eth_df=pd.DataFrame(eth_sentiments)

In [9]:
eth_df.sort_values(by=["compound"], ascending=False).head()

,text,date,compound,positive,negative,neutral
64,"""Within a decade, quantum computers could be p...",2021-06-14,0.8481,0.290,0.0,0.710
66,Disclosure: Our goal is to feature products an...,2021-06-21,0.7783,0.191,0.0,0.809
3,"Ether holders have ""staked"" more than $13 bill...",2021-07-05,0.7717,0.194,0.0,0.806
55,Institutional interest in cryptocurrencies lik...,2021-06-15,0.6705,0.164,0.0,0.836
89,What happened\r\nThe price of popular cryptocu...,2021-06-24,0.6369,0.140,0.0,0.860


In [10]:
# Order the DataFrame by highest avg score in descending order
btc_df.sort_values(by=["compound"], ascending=False).head()

,text,date,compound,positive,negative,neutral
88,Bitcoin\r\nNurphoto / Getty Images\r\nA swath ...,2021-07-10,0.7506,0.209,0.0,0.791
0,"It's all about clean energy, it seems. \r\nElo...",2021-06-14,0.6908,0.169,0.0,0.831
23,Posted \r\nA rare pear-shaped diamond that is ...,2021-06-21,0.5994,0.133,0.0,0.867
49,By Reuters Staff\r\nJune 13 (Reuters) - Tesla ...,2021-06-13,0.5859,0.134,0.0,0.866
69,* Graphic: World FX rates tmsnrt.rs/2RBWI5E\r\...,2021-06-21,0.5423,0.127,0.0,0.873


In [11]:
btc_avg=btc_df["positive"].mean()
btc_avg

0.03455

In [12]:
eth_avg=eth_df["positive"].mean()
eth_avg

0.04783999999999999

### Questions:

Q: Which coin had the highest mean positive score?

A: Ethereum had the higest mean positive score of .050

Q: Which coin had the highest compound score?

A: Ethereum had the highest compund score of .84

Q. Which coin had the highest positive score?

A: Ethereum had the higest positive score of .29

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [40]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [41]:
# Instantiate the lemmatizer
lemmatizer=WordNetLemmatizer()
wnl = WordNetLemmatizer()

# Create a list of stopwords
sw= []
sw = set(stopwords.words("English"))

# Expand the default stopwords list if necessary
sw_addon = {"TL;DR"}  ## custom for this article

In [42]:
def tokenizer(text):
    """Tokenizes text."""
    # Create a list of the words
    words = word_tokenize(text)
    # Convert the words to lowercase
    words = list(filter(lambda w: w.lower(), words))
    # Remove the punctuation
    words = list(filter(lambda t: t not in punctuation, words))
    # Remove the stopwords
    words = list(filter(lambda t: t.lower() not in sw, words))
    # Lemmatize Words into root words
    tokens = [wnl.lemmatize(word) for word in words]
    return tokens

In [43]:
# Create a new tokens column for Bitcoin
btc_df["texts_tok"] = btc_df["text"].apply(tokenizer)

In [44]:
# Create a new tokens column for Ethereum
eth_df["texts_tok"] =eth_df["text"].apply(tokenizer)

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [46]:
from collections import Counter
from nltk import ngrams

In [50]:
btc_list=[x for i in range(len(btc_df["texts_tok"])) for x in btc_df["texts_tok"] if str(x) not in punctuation]

In [52]:
# Generate the Bitcoin N-grams where N=2
btc_counts = Counter(ngrams(btc_list, n=2))
print(dict(btc_counts))

TypeError: unhashable type: 'list'

In [17]:
# Generate the Ethereum N-grams where N=2
# YOUR CODE HERE!

In [18]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [19]:
# Use token_count to get the top 10 words for Bitcoin
# YOUR CODE HERE!

In [20]:
# Use token_count to get the top 10 words for Ethereum
# YOUR CODE HERE!

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [21]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [22]:
# Generate the Bitcoin word cloud
# YOUR CODE HERE!

In [23]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [24]:
import spacy
from spacy import displacy

In [25]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [26]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [27]:
# Concatenate all of the Bitcoin text together
# YOUR CODE HERE!

In [28]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [29]:
# Render the visualization
# YOUR CODE HERE!

In [30]:
# List all Entities
# YOUR CODE HERE!

---

### Ethereum NER

In [31]:
# Concatenate all of the Ethereum text together
# YOUR CODE HERE!

In [32]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [33]:
# Render the visualization
# YOUR CODE HERE!

In [34]:
# List all Entities
# YOUR CODE HERE!

---